# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
geoapify_key = "192d02d301c14c55bde032ccf5d0408a"

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keflavik,64.0049,-22.5624,16.95,55,75,2.57,IS,1690484796
1,1,puerto natales,-51.7236,-72.4875,0.94,80,6,0.33,CL,1690484796
2,2,abepura,-2.6333,140.5833,24.45,83,40,1.03,ID,1690484796
3,3,hammerfest,70.6634,23.6821,10.25,81,75,3.09,NO,1690484797
4,4,kiunga,-6.1219,141.2906,22.29,99,95,0.96,PG,1690484797


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    alpha=0.5,
    frame_width=800,
    frame_height=400
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) 
                                    & (city_data_df["Max Temp"] < 27)
                                    & (city_data_df["Wind Speed"] < 4.5)
                                    & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
filtered_data_df.dropna()

# Display sample data
filtered_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,yamba,-5.9000,22.3000,24.85,26,0,1.87,CD,1690484799
57,57,cinque frondi,38.4162,16.0842,23.22,77,0,0.98,IT,1690484805
77,77,chibougamau,49.9168,-74.3659,21.79,37,0,2.66,CA,1690484562
234,234,sparti,37.0733,22.4297,26.67,57,0,1.48,GR,1690484847
276,276,tyul'gan,52.3405,56.1660,22.13,54,0,1.91,RU,1690484861


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,yamba,CD,-5.9000,22.3000,26,
57,cinque frondi,IT,38.4162,16.0842,77,
77,chibougamau,CA,49.9168,-74.3659,37,
234,sparti,GR,37.0733,22.4297,57,
276,tyul'gan,RU,52.3405,56.1660,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yamba - nearest hotel: No hotel found
cinque frondi - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
sparti - nearest hotel: No hotel found
tyul'gan - nearest hotel: No hotel found
oral - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
almeria - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
kirsanov - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,yamba,CD,-5.9000,22.3000,26,No hotel found
57,cinque frondi,IT,38.4162,16.0842,77,No hotel found
77,chibougamau,CA,49.9168,-74.3659,37,No hotel found
234,sparti,GR,37.0733,22.4297,57,No hotel found
276,tyul'gan,RU,52.3405,56.1660,54,No hotel found
305,oral,KZ,51.2333,51.3667,69,No hotel found
359,lodja,CD,-3.4833,23.4333,29,No hotel found
383,howard springs,AU,-12.4970,131.0470,67,No hotel found
469,almeria,ES,37.1667,-2.3333,56,No hotel found
509,vallenar,CL,-28.5708,-70.7581,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    alpha=0.5,
    frame_width=800,
    frame_height=400
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)